## House price calculator software

In [ ]:
import numpy as np
import pandas as pd
import sys
import json
import joblib
import importlib.util  
sys.path.append('./model')
sys.path.append('./preprocessing')
sys.path.append('./predict')
#import preprocessing.cleaning_data as cleaning_data
#import predict.prediction as prediction
from preprocessing.cleaning_data import checkData
from preprocessing.cleaning_data import creatingDummies
from predict.prediction import predict_price
#import preprocessing.cleaning_data as cleaning_data
#import predict.prediction as prediction 

from flask import Flask, request, jsonify, render_template

app = Flask(__name__)

@app.route('/', methods=['GET'])
def alive():
    return '<h1>This is server is alive!</h1>'


@app.route('/predict', methods=['POST','GET'])
def predict():
    """
    This function will be use for the prediction of property
    """
    
    if request.method == 'POST':
            number_rooms = request.form.get('number_rooms')
            living_area =  request.form.get('living_area')
            surface_area =  request.form.get('surface_area')
            number_facades =  request.form.get('number_facades')
            furnished = request.form.get('furnisheddropdown')
            terrace = request.form.get('terracedropdown')
            garden = request.form.get('gardendropdown')
            fireplace = request.form.get('fireplacedropdown')
            pool = request.form.get('pooldropdown')
            #REMEMBER: this is property subtype in de df
            property_type= request.form.get('propertydropdown')
            kitchen = request.form.get('kitchendropdown')
            condition = request.form.get('conditiondropdown')